# Importante
### Para la ejecución del presente notebook, es necesario descomprimir la carpeta .zip que se entrega junto al presente documento

---

# Entrega 2.2 - Consultas estructuradas

## Importar librerias

In [1]:
from glob import glob
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table, Column, Text
from sqlalchemy.types import Integer, Text, String, DateTime
from sqlalchemy.orm import Session

## Crear máquina para las consultas SQL

In [2]:
engine = create_engine('sqlite://')
meta = MetaData()
meta.bind = engine

## Cargar datasets

In [3]:
files = sorted(glob('end/part*'))

df_viajes = pd.read_parquet(files[0])
for f in files[1:]:
    df = pd.concat([df_viajes, pd.read_parquet(f)])
    
clima = sorted(glob('clima/*'))

df_clima = pd.read_csv(clima[0])
for g in clima[1:]:
    df_clima = pd.concat([df_clima, pd.read_csv(g)])

C:\Users\mpcor\AppData\Local\Temp\ipykernel_7464\2360169389.py:9: DtypeWarning: Columns (7,17,27,29,31,33,37,39,45,47,49,51,53,55,57,59,61,63,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clima = pd.read_csv(clima[0])
C:\Users\mpcor\AppData\Local\Temp\ipykernel_7464\2360169389.py:11: DtypeWarning: Columns (7,11,15,17,27,29,31,33,35,37,39,45,47,49,51,53,55,57,59,61,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clima = pd.concat([df_clima, pd.read_csv(g)])
C:\Users\mpcor\AppData\Local\Temp\ipykernel_7464\2360169389.py:11: DtypeWarning: Columns (13,17,23,29,31,53,55,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clima = pd.concat([df_clima, pd.read_csv(g)])


## Crear tablas para consultas SQL

In [6]:
df_viajes.to_sql('viajes', engine, if_exists='replace')
df_clima.to_sql('clima', engine, if_exists='replace')

229327

## Escribir codigo y sentencias para obtener las respuestas

**¿En qué semana, mes y en qué año fue el momento más caluroso? ¿Dónde?**

<i>Durante la semana del 25 al 31 de julio de 2020, fue el momento más caluroso con temperatura promedio de 32.21 en NY

In [7]:
query = """
WITH CTE AS(
SELECT AVG(TMAX) T_PROMEDIO_SEMANA, SEMANA, NAME, STATION FROM(
SELECT DISTINCT B.STATION, NAME, B.DATE, DATE(B.DATE,'weekday 5') SEMANA, B.TMAX FROM viajes A
INNER JOIN clima B ON A.END_WEATHER_STATION_JOIN=B.STATION AND A.end_date=B.DATE
WHERE TMAX<>'NaN')A
GROUP BY SEMANA, NAME
order by T_PROMEDIO_SEMANA DESC, SEMANA LIMIT 1)

SELECT T_PROMEDIO_SEMANA, DATE(SEMANA,'-6 days') || ' a ' || SEMANA SEMANA, STRFTIME('%m', SEMANA) MES, STRFTIME('%Y', SEMANA) AÑO, NAME LUGAR FROM CTE
"""
pd.read_sql(query, con=engine)

,T_PROMEDIO_SEMANA,SEMANA,MES,AÑO,LUGAR
0,32.214286,2020-07-25 a 2020-07-31,07,2020,"NY CITY CENTRAL PARK, NY US"


**¿Cuántas bicicletas fueron alquiladas esa semana más calurosa?**

<i>Durante esa semana, se alquilaron 872 bicicletas

In [8]:
query = """
WITH CTE AS(
SELECT AVG(TMAX) T_PROMEDIO_SEMANA, SEMANA, NAME, STATION FROM(
SELECT DISTINCT B.STATION, NAME, B.DATE, DATE(B.DATE,'weekday 5') SEMANA, B.TMAX FROM viajes A
INNER JOIN clima B ON A.END_WEATHER_STATION_JOIN=B.STATION AND A.end_date=B.DATE
WHERE TMAX<>'NaN')A
GROUP BY SEMANA, NAME
order by T_PROMEDIO_SEMANA DESC, SEMANA LIMIT 1)

SELECT COUNT(*) [Bicicletas alquiladas] FROM(
SELECT DISTINCT(bikeid) FROM CTE A INNER JOIN VIAJES B ON A.STATION=B.END_WEATHER_STATION_JOIN
WHERE B.END_DATE>=DATE(SEMANA,'-6 days') AND B.END_DATE<=SEMANA)A
"""
pd.read_sql(query, con=engine)

,Bicicletas alquiladas
0,872


**¿En qué semana, mes y en qué año fue el momento más frío?**

<i>Durante la semana del 20 de diciembre de 2017 al 5 de enero de 2018, fue el momento más frío con temperatura promedio de -10.54 en NY

In [9]:
query = """
WITH CTE AS(
SELECT AVG(TMIN) T_PROMEDIO_SEMANA, SEMANA, NAME, STATION FROM(
SELECT DISTINCT B.STATION, NAME, B.DATE, DATE(B.DATE,'weekday 5') SEMANA, B.TMIN FROM viajes A
INNER JOIN clima B ON A.END_WEATHER_STATION_JOIN=B.STATION AND A.end_date=B.DATE
WHERE TMIN<>'NaN')A
GROUP BY SEMANA, NAME
order by T_PROMEDIO_SEMANA, SEMANA LIMIT 1)

SELECT T_PROMEDIO_SEMANA, DATE(SEMANA,'-6 days') || ' a ' || SEMANA SEMANA, STRFTIME('%m', SEMANA) MES, STRFTIME('%Y', SEMANA) AÑO, NAME LUGAR FROM CTE
"""
pd.read_sql(query, con=engine)

,T_PROMEDIO_SEMANA,SEMANA,MES,AÑO,LUGAR
0,-10.542857,2017-12-30 a 2018-01-05,01,2018,"NY CITY CENTRAL PARK, NY US"


**¿Cuántas bicicletas fueron alquiladas esa semana?**

<i>Durante esa semana, se alquilaron 127 bicicletas

In [10]:
query = """
WITH CTE AS(
SELECT AVG(TMIN) T_PROMEDIO_SEMANA, SEMANA, NAME, STATION FROM(
SELECT DISTINCT B.STATION, NAME, B.DATE, DATE(B.DATE,'weekday 5') SEMANA, B.TMIN FROM viajes A
INNER JOIN clima B ON A.END_WEATHER_STATION_JOIN=B.STATION AND A.end_date=B.DATE
WHERE TMIN<>'NaN')A
GROUP BY SEMANA, NAME
order by T_PROMEDIO_SEMANA, SEMANA LIMIT 1)

SELECT COUNT(*) [Bicicletas alquiladas] FROM(
SELECT DISTINCT(bikeid) FROM CTE A INNER JOIN VIAJES B ON A.STATION=B.END_WEATHER_STATION_JOIN
WHERE B.END_DATE>=DATE(SEMANA,'-6 days') AND B.END_DATE<=SEMANA)A
"""
pd.read_sql(query, con=engine)

,Bicicletas alquiladas
0,127


## Preguntas de respuesta abierta (corta)

* ¿Cree usted que la diferencia entre bicicletas alquiladas tiene relación con el clima? 

<i>Considerando que ambos puntos de información comprenden temperaturas promedio tan extremas, es lógico pensar que el alquiler de bicicletas está relacionado con el clima.<br>En una misma ciudad, como NY, los climas durante verano y durante invierno pueden alcanzar niveles máximos y mínimos por lo que las personas tendrán mayor o menor disposición a alquilar bicicletas, sin contar con las condiciones de las vías disponibles para transitar en bicicleta en ambos momentos del año

* ¿Tiene sentido obtener una conclusión basados solo en un par de puntos de información?

<i>Con 2 puntos de información la conclusión puede ser incompleta o sesgada. A menos que el análisis se haga para la misma ciudad y con promedios sobre la misma semana y a lo largo de los años, sería posible sacar una conclusión a cerca del comportamiento de los alquileres de bicicleta con respecto al clima